In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/signapoop/Desktop/fyp-graph-clustering'

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import pickle 
import numpy as np
import pandas as pd
import torch
import os
from timeit import default_timer as timer
import networkx as nx
import scipy.sparse as sp

from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import pairwise_distances

In [3]:
from core.SimpleNet import SimpleNet
from core.OldGraphConvNet import OldGraphConvNet
from core.GraphConvNet import GraphConvNet
from core.EmbeddingDataSet import EmbeddingDataSet
from util.plot_graph_embedding import plot_graph_embedding
from util.evaluation_metrics import neighborhood_preservation, trustworthiness, evaluate_net_metrics
from util.network_utils import get_net_projection, get_graph_net_embeddings
from util.graph_utils import get_shortest_path_matrix

In [4]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
if torch.cuda.is_available():
    print('cuda available')
    device = 'cuda'
else:
    print('cuda not available')
    device = 'cpu'

cuda not available


In [6]:
dataset_name = 'cora_full'
parent_dir = os.path.abspath('..')
input_dir = os.path.join(parent_dir, 'data')
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)

In [7]:
net_parameters = {}
net_parameters['n_components'] = 2
net_parameters['D'] = dataset.input_dim # input dimension
net_parameters['H'] = 50 # number of hidden units
net_parameters['L'] = 2 # number of hidden layers

In [8]:
# Graph net
net_1 = GraphConvNet(net_parameters)
if torch.cuda.is_available(): net_1.cuda()
root = 'results/cora_26/'
filename = root + 'graph_net_420.pkl'
checkpoint = torch.load(filename, map_location=device)
net_1.load_state_dict(checkpoint['state_dict'])

In [9]:
dataset.create_all_data(n_batches=1, shuffle=False)
y_pred_1 = get_net_projection(dataset.all_data, net_1)
y_pred_1 = y_pred_1[np.argsort(dataset.all_indices)]
plot_graph_embedding(y_pred_1, dataset.labels, dataset.adj_matrix)

Time to create all data (s) = 0.0120


In [10]:
net_1.eval()

GraphConvNet(
  (gnn_cells): ModuleList(
    (0): GraphConvNetCell(
      (Ui1): Linear(in_features=1433, out_features=50, bias=False)
      (Uj1): Linear(in_features=1433, out_features=50, bias=False)
      (Vi1): Linear(in_features=1433, out_features=50, bias=False)
      (Vj1): Linear(in_features=1433, out_features=50, bias=False)
      (Ui2): Linear(in_features=50, out_features=50, bias=False)
      (Uj2): Linear(in_features=50, out_features=50, bias=False)
      (Vi2): Linear(in_features=50, out_features=50, bias=False)
      (Vj2): Linear(in_features=50, out_features=50, bias=False)
      (bn1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (R): Linear(in_features=1433, out_features=50, bias=False)
    )
  )
  (fc): Linear(in_features=50, out_features=2, bias=True)
)

In [11]:
embeddings = get_graph_net_embeddings(dataset.all_data, net_1)

In [17]:
embedder = TSNE(n_components=2, method="exact", perplexity=30, verbose=1)
y_pred_tsne = embedder.fit_transform(embeddings)

[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 1000 / 2708
[t-SNE] Computed conditional probabilities for sample 2000 / 2708
[t-SNE] Computed conditional probabilities for sample 2708 / 2708
[t-SNE] Mean sigma: 1.576432
[t-SNE] KL divergence after 250 iterations with early exaggeration: 67.806463
[t-SNE] KL divergence after 1000 iterations: 0.897170


In [18]:
plot_graph_embedding(y_pred_tsne, dataset.labels, dataset.adj_matrix)